In [1]:
import numpy as np
import pandas as pd
import geopandas as gd
import altair as alt
import unicodedata

---

### mapa_municipios.json cleaning

In [2]:
df_mapa = gd.read_file('../data/raw/mapa_municipios.json', encoding='utf-8')

# drop irrelevant data
df_mapa = df_mapa.drop(columns=['id', 'codigo_municipio_s'])

# rename columns
df_mapa.columns = ['id', 'geometry']

# correct data types
df_mapa['id'] = df_mapa['id'].astype(np.int64)

# setting the correct crs
df_mapa = df_mapa.set_crs("EPSG:4326").to_crs("EPSG:4326")

# adding area columns
df_mapa["area"] = df_mapa['geometry'].area

# df_mapa.sample(3)

/var/folders/5v/mphxd9d54wb59kvm9tlf4pj40000gn/T/ipykernel_2664/3460200609.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_mapa["area"] = df_mapa['geometry'].area


In [3]:
# df_mapa.info()

---

### metadata.csv cleaning

In [4]:
# read the data
df_metadata = gd.read_file('../data/raw/metadata.csv', encoding='utf-8')

# drop irrelevant data
startswith_remove = (
    "2",
    "Build",
    "TEMP",
    "PREC",
    "Numb",
    "Retire",
    "Age"
)
exact_remove = {
    "NumberofhospitalsperKm2",
    "Peopledoinghousework(%)",
    "PeoplewithDisabilities(%)",
    "AfrocolombianPopulation(%)",
    "IndianPopulation(%)",
}
df_metadata = df_metadata.loc[:, ~df_metadata.columns.str.startswith(startswith_remove)]
df_metadata = df_metadata.loc[:, ~df_metadata.columns.isin(exact_remove)]

# rename columns
df_metadata = df_metadata.rename(columns={
    'Municipality code': 'id',
    'Municipality': 'name',
    'Peoplewhocannotreadorwrite(%)': 'illiterate',
    'Secondary/HigherEducation(%)': 'education',
    'Employedpopulation(%)': 'employed',
    'Unemployedpopulation(%)': 'unemployed',
    'Men(%)': 'men',
    'Women(%)': 'women',
    'Householdswithoutwateraccess(%)': 'water',
    'Householdswithoutinternetaccess(%)': 'internet'
})
df_metadata.columns = [
    col.replace("Population", "pop")
    if col.startswith("Population") else col
    for col in df_metadata.columns
]
df_metadata.columns = df_metadata.columns.str.lower()

# correcting data types
float_cols = [
    "illiterate", "education", "employed", "unemployed",
    "men", "women", "water", "internet"
]

int_cols = [
    "id", "pop2007", "pop2008", "pop2009", "pop2010",
    "pop2011", "pop2012", "pop2013", "pop2014", "pop2015",
    "pop2016", "pop2017", "pop2018", "pop2019",
    "cases2007", "cases2008", "cases2009", "cases2010",
    "cases2011", "cases2012", "cases2013", "cases2014",
    "cases2015", "cases2016", "cases2017", "cases2018", "cases2019"
]
df_metadata[float_cols] = df_metadata[float_cols].astype("float64")
df_metadata[int_cols] = df_metadata[int_cols].astype("int64")

# df_metadata.sample(3)

In [5]:
# df_metadata.info()

---

### master dataset creation

In [6]:
# merge geometry and metadata
df_master = df_mapa.merge(df_metadata, on="id", how="left")

# reordering columns
columns_order = [
    "id", "name", "illiterate", "education", "employed", "unemployed",
    "water", "internet", "men", "women",
    "pop2007", "pop2008", "pop2009", "pop2010", "pop2011", "pop2012",
    "pop2013", "pop2014", "pop2015", "pop2016", "pop2017", "pop2018", "pop2019",
    "cases2007", "cases2008", "cases2009", "cases2010", "cases2011", "cases2012",
    "cases2013", "cases2014", "cases2015", "cases2016", "cases2017", "cases2018", "cases2019",
    "area", "geometry"
]
df_master = df_master.reindex(columns=columns_order)

# df_master.sample(3)

In [7]:
# df_master.info()

---

### map check

In [8]:
# # shapes
# (
#     alt
#     .Chart(df_master)
#     .mark_geoshape()
#     .encode(
#         tooltip=['municipal:N']
#     )
#     .project(
#         type='mercator'
#     )
#     .properties(
#         width=800,
#         height=800
#     )
# )

In [9]:
# # shapes
# temp = df_master.copy()

# # population density
# temp["density2019"] = df_master["pop2019"] / df_master["area"]

# (
#     alt.Chart(temp)
#     .mark_geoshape()
#     .encode(
#         tooltip=["name:N", 'pop2019:Q', 'area:Q'],
#         color=alt.Color("density2019:Q", scale=alt.Scale(type="log", scheme="reds"), title='People per km²')
#     )
#     .project(type="mercator")
#     .properties(width=600, height=600, title='Colombia 2019 Population Density')
# )

---

### export to separate files

In [ ]:
# df_metadata.to_csv('../data/meta.csv', index=None)
# df_mapa.to_file('../data/geo.geojson', driver='GeoJSON')
# df_master.to_file('../data/master.geojson', driver='GeoJSON')